# Model creation and data split-up

This notebook will:
* Train the model
* Save the model and predictions

Import packages

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pds
import datetime as dt
from collections import deque
import random

Import data

In [2]:
df_tr = pds.read_pickle('ANN_train_203_downsampled.pkl')
df_te = pds.read_pickle('ANN_test_203_downsampled.pkl')
df = pds.concat([df_tr, df_te], keys=['train', 'test'])
df.head()

direction  journey       lat       lon  seg     speed  \
train 0  129.852944        1  0.325030  0.926476    1  0.051520   
      1  142.963046        1  0.329938  0.928692    1  0.210201   
      2  287.399994        1  0.326437  0.944423    1  0.276662   
      3  289.790009        1  0.322491  0.946723    1  0.398248   
      4  288.200012        1  0.320008  0.948639    1  0.427615   

                        timestamp  time_left  segment_time  tsjs   lat_one  \
train 0 2018-02-16 04:48:46+01:00       65.0          65.0   0.0  0.325030   
      1 2018-02-16 04:49:05+01:00       46.0          65.0   0.0  0.325030   
      2 2018-02-16 04:49:17+01:00       34.0          65.0   0.0  0.329938   
      3 2018-02-16 04:49:18+01:00       33.0          65.0   0.0  0.326437   
      4 2018-02-16 04:49:19+01:00       32.0          65.0   0.0  0.322491   

          lon_one   lat_two   lon_two  lat_three  lon_three  speed_one  \
train 0  0.926476  0.325030  0.926476   0.325030   0.926476   0.325030   
      1  0.926476  0.325030  0.926476   0.325030   0.926476   0.325030   
      2  0.928692  0.325030  0.926476   0.325030   0.926476   0.329938   
      3  0.944423  0.329938  0.928692   0.325030   0.926476   0.326437   
      4  0.946723  0.326437  0.944423   0.329938   0.928692   0.322491   

         speed_two  speed_three  
train 0   0.325030     0.325030  
      1   0.325030     0.325030  
      2   0.325030     0.325030  
      3   0.329938     0.325030  
      4   0.326437     0.329938

One hot encode segments, normalize columns. time_left is the label in this model.

In [3]:
dataset = pds.DataFrame()
# Make segments categorical
dataset = pds.get_dummies(df['seg'])

# Combine segments, timestamps and time from journey start

# Cyclical timestamps 

dataset['hr_sin'] = np.sin((pds.to_datetime(df.timestamp).dt.hour*3600+pds.to_datetime(df.timestamp).dt.minute*60+pds.to_datetime(df.timestamp).dt.second)*(2.*np.pi/86400))
dataset['hr_cos'] = np.cos((pds.to_datetime(df.timestamp).dt.hour*3600+pds.to_datetime(df.timestamp).dt.minute*60+pds.to_datetime(df.timestamp).dt.second)*(2.*np.pi/86400))

# Convert to radians befor trigonometric functions
dataset['dir_sin'] = np.sin(df.direction*(np.pi/180))
dataset['dir_cos'] = np.cos(df.direction*(np.pi/180))

dataset['speed'] = df['speed']
# Previously was 'time_since_journey_start'
dataset['tsjs'] = (df['tsjs']-df['tsjs'].min())/(df['tsjs'].max()-df['tsjs'].min())

dataset['lat'] = df['lat']
dataset['lon'] = df['lon']

# Include journey number to select entire journeys
dataset['journey'] = df['journey']

dataset['time_left'] = df['time_left']



dataset.head()

1  2  3  4  5  6  7  8  9  10    ...        hr_sin    hr_cos  \
train 0  1  0  0  0  0  0  0  0  0   0    ...      0.952085  0.305834   
      1  1  0  0  0  0  0  0  0  0   0    ...      0.952507  0.304518   
      2  1  0  0  0  0  0  0  0  0   0    ...      0.952772  0.303687   
      3  1  0  0  0  0  0  0  0  0   0    ...      0.952794  0.303617   
      4  1  0  0  0  0  0  0  0  0   0    ...      0.952816  0.303548   

          dir_sin   dir_cos     speed  tsjs       lat       lon  journey  \
train 0  0.767692 -0.640819  0.051520   0.0  0.325030  0.926476        1   
      1  0.602330 -0.798247  0.210201   0.0  0.329938  0.928692        1   
      2 -0.954240  0.299041  0.276662   0.0  0.326437  0.944423        1   
      3 -0.940940  0.338574  0.398248   0.0  0.322491  0.946723        1   
      4 -0.949972  0.312335  0.427615   0.0  0.320008  0.948639        1   

         time_left  
train 0       65.0  
      1       46.0  
      2       34.0  
      3       33.0  
      4       32.0  

[5 rows x 21 columns]

Divide training/validation/test data set

### If the data is not split into train and test, this chunk chould be run
Roughly 20% Test and 20% Validation
num_test_segments = 700
num_validation_segments = 700
unique_journeys = dataset.journey.unique()
random.shuffle(unique_journeys)

msk1 = unique_journeys[0:num_test_segments]
msk2 = unique_journeys[num_test_segments:num_test_segments+num_validation_segments]
msk3 = unique_journeys[num_test_segments+num_validation_segments:len(unique_journeys)]
mask1 = dataset['journey'].isin(msk1)
mask2 = dataset['journey'].isin(msk2)
mask3 = dataset['journey'].isin(msk3)

In [4]:
test_input = dataset.loc['test']
#validation_input = dataset[mask2]
train_input = dataset.loc['train']


In [5]:
# Journey numbers were needed up until this point
train_data = train_input.drop(columns=['journey'])
test_data = test_input.drop(columns=['journey'])
#validation_data = validation_input.drop(columns=['journey'])

train_data.head()

,1,2,3,4,5,6,7,8,9,10,11,hr_sin,hr_cos,dir_sin,dir_cos,speed,tsjs,lat,lon,time_left
0,1,0,0,0,0,0,0,0,0,0,0,0.952085,0.305834,0.767692,-0.640819,0.051520,0.0,0.325030,0.926476,65.0
1,1,0,0,0,0,0,0,0,0,0,0,0.952507,0.304518,0.602330,-0.798247,0.210201,0.0,0.329938,0.928692,46.0
2,1,0,0,0,0,0,0,0,0,0,0,0.952772,0.303687,-0.954240,0.299041,0.276662,0.0,0.326437,0.944423,34.0
3,1,0,0,0,0,0,0,0,0,0,0,0.952794,0.303617,-0.940940,0.338574,0.398248,0.0,0.322491,0.946723,33.0
4,1,0,0,0,0,0,0,0,0,0,0,0.952816,0.303548,-0.949972,0.312335,0.427615,0.0,0.320008,0.948639,32.0


Modify dataset to contain sequences. This is required for the RNN. Each sequence is SEQUENCE_LENGTH consecutive data points. Currently a sequence can contain data points from two different journeys, this is not optimal. Should check if the sequence starts and ends in the same journey. This will also use a lot of RAM since the data is effectively increased by SEQUENCE_LENGTH times.

In [6]:
SEQUENCE_LENGTH = 20
def sequencify(df, shuffle):
    sequencial_data = []
    sequence = deque(maxlen=SEQUENCE_LENGTH)
    for i in df.values:
        sequence.append([n for n in i[:-1]])
        if len(sequence) == SEQUENCE_LENGTH:
            sequencial_data.append([np.array(sequence), i[-1]])
    if shuffle:
        random.shuffle(sequencial_data)
    X = []
    Y = []
    for sequence, label in sequencial_data:
        X.append(sequence)
        Y.append(label)
    return np.array(X),Y

Creating the sequences of training data, test data and validation data. Test data is not randomized in order to be able to plot it in a nice way.

In [7]:
seq_train_x, seq_train_y = sequencify(train_data, True)
seq_test_x, seq_test_y = sequencify(test_data, False)
#seq_validation_x, seq_validation_y = sequencify(validation_data, True)

Model creation

In [8]:
BATCH_SIZE = 64
EPOCHS = 8
model = keras.Sequential()
model.add(keras.layers.CuDNNLSTM(128, input_shape=(seq_train_x.shape[1:]), return_sequences=True))

model.add(keras.layers.CuDNNLSTM(128, input_shape=(seq_train_x.shape[1:]), return_sequences=True))

model.add(keras.layers.CuDNNLSTM(128, input_shape=(seq_train_x.shape[1:])))

model.add(keras.layers.Dense(32, activation="relu"))

model.add(keras.layers.Dense(1, activation="relu"))

opt = keras.optimizers.Adadelta()

model.compile(loss='mae',optimizer=opt, metrics=['mae'])

Fit model

In [9]:
result = model.fit(seq_train_x,seq_train_y,
                  batch_size=BATCH_SIZE,
                  epochs=EPOCHS,
                  validation_split=0.1, )

Train on 1354808 samples, validate on 150535 samples
Epoch 1/8
1354808/1354808 [==============================] - 212s 156us/step - loss: 3.9177 - mean_absolute_error: 3.9177 - val_loss: 3.1965 - val_mean_absolute_error: 3.1965
Epoch 2/8
1354808/1354808 [==============================] - 210s 155us/step - loss: 3.1265 - mean_absolute_error: 3.1265 - val_loss: 3.0726 - val_mean_absolute_error: 3.0726
Epoch 3/8
1354808/1354808 [==============================] - 212s 156us/step - loss: 3.0312 - mean_absolute_error: 3.0312 - val_loss: 2.9607 - val_mean_absolute_error: 2.9607
Epoch 4/8
1354808/1354808 [==============================] - 210s 155us/step - loss: 2.9747 - mean_absolute_error: 2.9747 - val_loss: 2.9641 - val_mean_absolute_error: 2.9641
Epoch 5/8
1354808/1354808 [==============================] - 211s 156us/step - loss: 2.9389 - mean_absolute_error: 2.9389 - val_loss: 2.9343 - val_mean_absolute_error: 2.9343
Epoch 6/8
1354808/1354808 [==============================] - 207s 153us/

Evaluate model on all test data

In [10]:
 model.evaluate(seq_test_x, seq_test_y)

375257/375257 [==============================] - 42s 112us/step


[3.547643580468715, 3.547643580468715]

Prepare data for the evaluation step

In [11]:
predictions = model.predict(seq_test_x)

### Create dataset for evaluation

In [14]:
segment = df['seg'].loc['test'].reset_index(drop=True)

In [18]:
output = pds.DataFrame()
output['seg'] = segment[19:]
output['journey'] = test_input['journey'][19:].values
output['speed'] = test_data['speed'][19:].values
output['pred'] = predictions
output['label'] = test_data['time_left'][19:].values
output = output.reset_index(drop = True)
output.head()

,seg,journey,speed,pred,label
0,1,12,0.671067,17.630503,20.0
1,1,12,0.678218,16.537560,19.0
2,1,12,0.680968,15.687718,18.0
3,1,12,0.684268,14.782970,17.0
4,1,12,0.689219,13.821644,16.0


In [19]:
# Same name for all files
model_name = 'ANN_M4_203_downsampled_benchmark'

# Serialize model to YAML
model_yaml = model.to_yaml()
with open(model_name + '.yaml', "w") as yaml_file:
    yaml_file.write(model_yaml)

# Serialize weights to HDF5
model.save_weights(model_name + '.h5')

In [20]:
output.to_pickle(model_name + '.pkl')

Done! Get ready for evaluation